In [5]:
import json
import requests
from tqdm import tqdm_notebook as tqdm

In [6]:
headers = {
    "Content-Type": "application/json",
    'accept': 'application/json'
}

In [7]:
def json_load(name):
    with open(f'{name}.json', 'r', encoding = 'utf-8') as f:
        return json.load(f)
    
def json_save(name, item):
    with open(f'{name}.json', 'w', encoding = 'utf-8') as f:
        json.dump(item, f, ensure_ascii = False, indent = 2)

In [14]:
main = json_load("../processed_data/QALD/qald-test-qa-dataset")[0]

In [15]:
def precision_at_k(data, k=1):
    # TODO: implement @k
    assert k > 0
    cnt = 0
    for i in list(data.keys()):
        if len(data[i]) > 0:
            if data[i][0][0] == 1:
                cnt += 1
    return cnt / len(list(data.keys()))

In [16]:
precision_at_k(main) # for initial set

0.02

In [17]:
# filter answer candidates and create new dataset for evaluation

new_main = dict()

for i in tqdm(list(main.keys())):
    answers = list()
    tmp = list()
    for tup in main[i]: # build q-a tuples set for a question
        tmp.append([
            tup[1][0], # question
            tup[1][1] # answer
        ])
    
    if len(tmp) > 0:
        data = json.dumps(tmp)
        json_response = requests.post('http://webengineering.ins.hs-anhalt.de:41003/predict',
                                      data=data,
                                       headers=headers)

        preds = json_response.json()['predictions'] # get predictions for the q-a tuples set

        for j in range(len(main[i])): # add only positive results to the new dataset
            if preds[j] == 1:
                answers.append(main[i][j])
            
    new_main[i] = answers

/data/benchmarking_RuBQ/benchmarking_RuBQ-env/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [18]:
precision_at_k(new_main, k=1) # for filtered set

0.02666666666666667